In [2]:
n=13
R.<x> = PolynomialRing(QQ)
P = [0*x+1, x+1]
# print(P[0], P[1])
for i in range(2,n):
    p=(P[i-1])^2+x*(P[i-2])^4
    P.append(p)
print(P[0], ' ', P[1], ' ', P[2], ' ', P[3])
print(P)

1   x + 1   x^2 + 3*x + 1   x^5 + 5*x^4 + 12*x^3 + 15*x^2 + 7*x + 1
[1, x + 1, x^2 + 3*x + 1, x^5 + 5*x^4 + 12*x^3 + 15*x^2 + 7*x + 1, x^10 + 11*x^9 + 61*x^8 + 208*x^7 + 452*x^6 + 627*x^5 + 547*x^4 + 292*x^3 + 91*x^2 + 15*x + 1, x^21 + 21*x^20 + 220*x^19 + 1523*x^18 + 7775*x^17 + 30925*x^16 + 98704*x^15 + 256560*x^14 + 545940*x^13 + 950028*x^12 + 1345112*x^11 + 1537951*x^10 + 1407161*x^9 + 1019787*x^8 + 578680*x^7 + 253652*x^6 + 84407*x^5 + 20815*x^4 + 3668*x^3 + 435*x^2 + 31*x + 1, x^42 + 43*x^41 + 925*x^40 + 13256*x^39 + 142124*x^38 + 1213323*x^37 + 8569167*x^36 + 51346436*x^35 + 265610327*x^34 + 1200837799*x^33 + 4786922029*x^32 + 16932184424*x^31 + 53385028776*x^30 + 150505312544*x^29 + 380220415128*x^28 + 861875056184*x^27 + 1754176822988*x^26 + 3206207502532*x^25 + 5261162612572*x^24 + 7745850043832*x^23 + 10222167934560*x^22 + 12077340137771*x^21 + 12755848302987*x^20 + 12022767095116*x^19 + 10092420542229*x^18 + 7528476229093*x^17 + 4977830422843*x^16 + 2909059303088*x^15 + 149

In [2]:
# Note that E represents the encoding that we are testing it out for. We know that for |E| <= 5, the statement holds by raw computation.
# What about when |E| >= 6? The purpose here is to generate different permutations of E such that the root is included.
# We will then generate E' with E and also run computations with E' such that we get different coefficients in the polynomials.

In [39]:
# Note that we have the current recurrence relation to determine the number of subtrees obtained from a given encoding as follows:
# F(n,e) = 0, if n = 0
# F(n,e) = 2F(n - 2, e[1:]), if e[0] = 1
# F(n,e) = F(n - 1, e[1:]), if e[0] = 0
# Then, we can turn that recurrence relation into a generating function polynomial given by:
# P_{n,e}(x)= \prod_{i=1}^{n} [(1-e_i)*( P_{n-i}(x) )+(e_i)*x*[P_{n-(i+1)}(x)]^2]
# Where the end levels of the base case is: P_{-1} =P_{0} = 1 for computations on last levels (i=n, n-1).

R.<x> = PolynomialRing(QQ)

# # Base cases for P(-1) and P(0)
# def P(n, e):
#     if n == -1 or n == 0:
#         return R(1)

#     result = R(1)
#     for i in range(1, n + 1):
#         term1 = (1 - e[i-1]) * P(n - i, e) # Recursive call #1
#         term2 = e[i-1] * x * (P(n - (i + 1), e))^2 # Recursive call #2
#         result *= (term1 + term2)

#     return result

def P_n_e(n, e):
    if n == -1 or n == 0: # for P[-1] = P[0] = 1
        return R(1)

    product = R(1)

    for i in range(1, n+1):
        e_i = e[i-1]

        if e_i == 0:
            product *= P[n-i]
        elif i <= n - 1:
            product *= x * (P[n-(i + 1)])^2
        elif i == n:
            product *= x

    return product

x^4 + 4*x^3 + 4*x^2 + x

In [3]:
def generate_encodings(n, current_encoding=None, encodings=None):
    if current_encoding is None:
        current_encoding = [1]

    if encodings is None:
        encodings = [] 
    
    if len(current_encoding) == n: 
        encodings.append(current_encoding)
        return encodings
    
    last_bit = current_encoding[-1]
    
    # If the last bit is 1, the next bit must be 0
    if last_bit == 1:
        generate_encodings(n, current_encoding + [0], encodings)
    # If the last bit is 0, the next bit can be either 0 or 1
    else:
        generate_encodings(n, current_encoding + [0], encodings)
        generate_encodings(n, current_encoding + [1], encodings)
    
    return encodings

In [27]:
n = 3
valid_encodings = generate_encodings(n)
print(valid_encodings)

[[1, 0, 0], [1, 0, 1]]


In [72]:
# We will then calculate the augmented polynomial given by the equation:
# P_{n,e}(x)= \prod_{i=1}^{n} [(1-e_i)*( P_{n-i}(x) )+(e_i)*x*[P_{n-(i+1)}(x)]^2]
# We will then check for E and E' and hopefully prove that our hypothesis that P_{n,E'}(x) - P_{n,E}(x) >= 0

n = 11
valid_encodings = generate_encodings(n)
filtered_encodings = []

for v in valid_encodings:
    if(v[0] != v[len(v) - 1]):
       filtered_encodings.append(v)

for e in filtered_encodings:
    # print(f"Encoding: {e}")
    forward_result = P_n_e(n, e)
    # print(forward_result)
    # print(f"Encoding: {e[::-1]}")x
    reverse_result = P_n_e(n, e[::-1])
    # print(reverse_result)
    difference = reverse_result - forward_result
    diff_list = list(difference)
    flag = true
    for i in diff_list:
        if(i < 0):
            flag = false
            break

    if(not flag):
        print(f"Encoding: {e}")
        print(f"Reversed encoding: {e[::-1]}")
        print("NOT OKAY")

Encoding: [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
Reversed encoding: [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1]
NOT OKAY
Encoding: [1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0]
Reversed encoding: [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1]
NOT OKAY
Encoding: [1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0]
Reversed encoding: [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1]
NOT OKAY


In [3]:
# Set up the polynomial ring in SageMath
R.<x> = PolynomialRing(QQ)

# Define the recurrence relations for P_n(x)
def generate_P_sequence(n):
    P = [1, x + 1]
    for i in range(2, n):
        p = P[i-1]^2 + x * P[i-2]^4
        P.append(p)
    return P

# Generate the sequence of P up to a reasonable depth
n = 10
P = generate_P_sequence(n)

# Function to calculate the counts for (P_{d-2})^3 and P_{d-1}(P_{d-3})^2
def calculate_coclique_counts(d):
    # Calculate Expression 1: (P_{d-2})^3 reduces to (P_{d-4})^4 (P_{d-2})^2
    expr1_count = 4 * count_cocliques(P[d-4]) + 2 * count_cocliques(P[d-2])
    
    # Calculate Expression 2: P_{d-1} (P_{d-3})^2 reduces to (P_{d-3})^6
    expr2_count = 6 * count_cocliques(P[d-3])
    
    return expr1_count, expr2_count

# Define the function to count cocliques from the given polynomial
def count_cocliques(polynomial):
    return len(polynomial.coefficients())

# Compare the counts for d = 5 (odd) and d = 6 (even)
print("Case 1: d = 5 (odd)")
expr1_count_odd, expr2_count_odd = calculate_coclique_counts(5)
print(f"Expression 1 Count: {expr1_count_odd}, Expression 2 Count: {expr2_count_odd}")
print(f"Injective function exists: {expr1_count_odd < expr2_count_odd}\n")

print("Case 2: d = 6 (even)")
expr1_count_even, expr2_count_even = calculate_coclique_counts(6)
print(f"Expression 1 Count: {expr1_count_even}, Expression 2 Count: {expr2_count_even}")
print(f"Injective function exists: {expr1_count_even < expr2_count_even}")


Case 1: d = 5 (odd)
Expression 1 Count: 20, Expression 2 Count: 18
Injective function exists: False

Case 2: d = 6 (even)
Expression 1 Count: 34, Expression 2 Count: 36
Injective function exists: True
